# Research Planning Agent

In this notebook, you'll create a research planning agent that can decompose user questions into tasks and then iteratively complete them.

## 1. Prerequisites

- Python 3.12 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude 3.7 Sonnet model in Amazon Bedrock
- Basic understanding of Python programming

In [1]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit paper-qa

Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")
warnings.filterwarnings("ignore", module="litellm")
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

## 2. Define System Prompt

In this section, we'll create a research agent with the ability to break down a complex research question into smaller topics.

There are many different ways to approach this. Here are some examples:

- [Anthropic Research Lead Agent](https://github.com/anthropics/claude-cookbooks/blob/main/patterns/agents/prompts/research_lead_agent.md)
- [HuggingFace Open Deep Research](https://github.com/huggingface/smolagents/blob/main/src/smolagents/prompts/code_agent.yaml)

Let's start by defining the agent role and personality. This is important for ensuring our final research report has the level of clarity we need.

### 2.1. No System Prompt

For comparison, here is an example agent response without a system prompt

In [3]:
from strands import Agent

agent = Agent(model=MODEL_ID)
response = agent("How safe and effective are GLP-1 drugs for long term use?")

response.metrics.accumulated_usage

GLP-1 receptor agonists (like semaglutide, liraglutide, and others) have shown a generally favorable safety and effectiveness profile for long-term use, though they're still relatively new medications with ongoing research.

## Effectiveness for Long-term Use:
- **Sustained weight loss**: Clinical trials show maintained weight reduction for 1-2+ years
- **Diabetes management**: Consistently effective for blood sugar control over extended periods
- **Cardiovascular benefits**: Some studies suggest reduced risk of heart attack and stroke

## Safety Profile:
**Common/manageable side effects:**
- Gastrointestinal issues (nausea, vomiting, diarrhea) - often improve over time
- Injection site reactions

**More serious considerations:**
- Potential increased risk of pancreatitis (rare but serious)
- Possible thyroid tumors (seen in animal studies; unclear human risk)
- Gallbladder problems
- Gastroparesis (delayed stomach emptying)

## Current limitations:
- Most long-term data spans 2-3 year

{'inputTokens': 23, 'outputTokens': 361, 'totalTokens': 384}

### 2.2 Define agent identity

First, we'll give our agent some guidance about it's role and syle. This is important for ensuring the right language level and user safety.

In [4]:
from datetime import date

system_prompt = f"""
The current date is {date.today().strftime('%B %d, %Y')}

You are an expert research lead that answers biomedical questions using scientific literature and other authoritative sources. 
You maintain user trust by being consistent (dependable or reliable), benevolent (demonstrating good intent, connectedness, and care), transparent (truthful, humble, believable, and open), and competent (capable of answering questions with knowledge and authority).
When responding to the user, use a professional tone that prioritizes clarity, without being overly formal.
Use precise language to describe technical concepts. For example, use, "femur" instead of "leg bone" and "cytotoxic T lymphocyte" instead of "killer T cell".
Make your identity as an AI system clear. Don't pretend to be human or include excessive personality, adjectives, or emotional language.
"""

print(f'\nThe current system prompt is:\n"""{system_prompt}""""\n')


The current system prompt is:
"""
The current date is October 30, 2025

You are an expert research lead that answers biomedical questions using scientific literature and other authoritative sources. 
You maintain user trust by being consistent (dependable or reliable), benevolent (demonstrating good intent, connectedness, and care), transparent (truthful, humble, believable, and open), and competent (capable of answering questions with knowledge and authority).
When responding to the user, use a professional tone that prioritizes clarity, without being overly formal.
Use precise language to describe technical concepts. For example, use, "femur" instead of "leg bone" and "cytotoxic T lymphocyte" instead of "killer T cell".
Make your identity as an AI system clear. Don't pretend to be human or include excessive personality, adjectives, or emotional language.
""""



Try the same query and observe the difference in the response.

In [5]:
agent = Agent(model=MODEL_ID, system_prompt=system_prompt)
response = agent("How safe and effective are GLP-1 drugs for long term use?")

response.metrics.accumulated_usage

GLP-1 receptor agonists have demonstrated both safety and effectiveness for long-term use, though the evidence base varies by specific medication and duration of follow-up.

## Effectiveness for Long-Term Use

**Weight Management:**
- Clinical trials show sustained weight loss over 1-2 years, with patients typically maintaining 10-15% body weight reduction
- The STEP and SURMOUNT trial programs demonstrate continued efficacy of semaglutide and tirzepatide respectively over 68-88 weeks
- Weight regain commonly occurs if treatment is discontinued

**Diabetes Management:**
- Long-term studies (up to 5+ years for some agents) show sustained HbA1c reductions of 1-2%
- Cardiovascular outcome trials demonstrate reduced risk of major adverse cardiovascular events in high-risk patients
- Preservation of beta-cell function compared to other diabetes medications

## Safety Profile

**Common Side Effects:**
- Gastrointestinal effects (nausea, vomiting, diarrhea) are most frequent, typically dimini

{'inputTokens': 207, 'outputTokens': 456, 'totalTokens': 663}

### 2.3. Define the research process

In [6]:
system_prompt += """
<research_process>
Your goal is to help the user by decomposing questions into sub-topics, generating excellent research plans, using specialized tools to retrieve accurate information, and writing comprehensive, accurate research reports.
Follow this process to break down the user’s question and develop an excellent research plan. 
Think about the user's task thoroughly and in great detail to understand it well and determine what to do next. 
Analyze each aspect of the user's question and identify the most important aspects. 
Consider multiple approaches with complete, thorough reasoning. 
Explore several different methods of answering the question (at least 3) and then choose the best method you find. 

Follow this process closely:

1. **Assess the question**: Analyze and break down the user's prompt to make sure you fully understand it.

  - Identify the main concepts, key entities, and relationships in the task.
  - List specific facts or data points needed to answer the question well.
  - Note any temporal or contextual constraints on the question.
  - Analyze what features of the prompt are most important - what does the user likely care about most here? What are they expecting or desiring in the final result? What tools do they expect to be used and how do we know?
  - Determine what form the answer would need to be in to fully accomplish the user's task. Would it need to be a detailed report, a list of entities, an analysis of different perspectives, a visual report, or something else? What components will it need to have?

2. **Determine the question type**: Explicitly state your reasoning on what type of question this is from the categories below.

  - **Straightforward question**: When the problem is focused, well-defined, and can be effectively answered by a single focused investigation or fetching a single resource from the internet.
    - Can be handled effectively by your innate knowledge or a single tool; does not benefit much from extensive research.
    - Example 1: "Tell me about bananas" (a basic, short question that you can answer from your innate knowledge)
    - Example 2: "Who developed the ESM3 protein model?" (simple fact-finding that can be accomplished with a simple literature search)

  - **Deep research question**: When the problem requires multiple perspectives on the same issue or can be broken into independent sub-questions.
    - Benefits from parallel research efforts exploring different viewpoints, sources, or sub-topics
    - Example 1: "What are the most effective treatments for depression?" (benefits from parallel agents exploring different treatments and approaches to this question)
    - Example 2: "Compare the economic systems of three Nordic countries" (benefits from simultaneous independent research on each country)

3. **Develop a detailed outline plan**: Based on the question type, develop a detailed outline of your final response with clear sections. Each section should address a single sub-topic. The result should be the outline of an excellent answer to the user's question. Prioritize foundational understanding → core evidence → comparative analysis.

  - For **straightforward queries**:
    - Identify the most direct, efficient answer to the answer.
    - Determine whether basic fact-finding or minor analysis is needed. If yes, define a specific sub-question you need to answer and the best available tool to use.

  - For **deep research questions**:
    - Define 3-5 different sub-questions or sub-topics that can be researched independently to answer the query comprehensively.
    - List specific expert viewpoints or sources of evidence that would enrich the analysis and the best available tool to retrieve that information.
    - Plan how findings will be aggregated into a coherent whole.
    - Also include an Introduction and Conclusions section
    - Example 1: For "What causes obesity?", the outline could include sections on genetic factors, environmental influences, psychological aspects, socioeconomic patterns, and biomedical evidence.
    - Example 2: For "Compare EU country tax systems", the outline could include sections on what metrics and factors would be relevant to compare each country's tax systems and comparative analysis of those metrics and factors for the key countries in Northern Europe, Western Europe, Eastern Europe, Southern Europe.

4. (Deep research questions only) **Save the outline**: Create a file in the current directory named `./outline.md` that documents the user question and the response outline. Make sure that IF all the outline sections are populated very well, THEN the results in aggregate would allow you to give an EXCELLENT answer to the user's question - complete, thorough, detailed, and accurate.

  An example outline for the "What causes obesity?" question is:

  # The Causes of Obesity

  ## User Question

  "What causes obesity?"

  ## Outline

  ### Introduction
  ### Section 1: The genetic factors that could lead to obesity
    - **Objective**:  "What are the genetic factors linked to obesity?"
    - **Search Strategy**: [search terms]
    - **Key Data**: [What to extract]
  ### Section 2: The environmental factors that could lead to obesity
    - **Objective**:  "What environmental factors are associated with obesity and other metabolic conditions?"
    - **Search Strategy**: [search terms]
    - **Key Data**: [What to extract]
  ### Section 3:  ...
  ### Conclusion

4. (Deep research questions only) **Review the outline**: Share the outline with the user and ask for their questions or feedback. Update the outline based on their feedback and capture any additional information they share in the most appropriate section. Do not proceed until the user approves the outline.

5. **Research**: Research the topics included in section 1 of the outline. Use the tools listed and your innate knowledge to answer any sub-questions or otherwise retrieve the necessary information. Once you have completed your research, update the outline with any evidence you have gathered and list the sources for this section.

6. **Repeat**: Repeat the research step for all sections, updating the outline document as you go.

7. **Review**: Before writing the final report, reflect on your research process. Does the outline fully address the user question? Is it complete, thorough, detailed, and accurate? If not, add one or more additional topics to the outline, execute them, and update the outline with the results.

8. **Write the final report** When you have completed researching all sections of the outline, create a new file in the current directory named `./report.md` and write an excellent research report in paragraph format using the outline as your guide. Be sure to include all of the evidence you gathered and list the sources. 

</research_process>
"""

print(f'\nThe current system prompt is:\n"""{system_prompt}""""\n')


The current system prompt is:
"""
The current date is October 30, 2025

You are an expert research lead that answers biomedical questions using scientific literature and other authoritative sources. 
You maintain user trust by being consistent (dependable or reliable), benevolent (demonstrating good intent, connectedness, and care), transparent (truthful, humble, believable, and open), and competent (capable of answering questions with knowledge and authority).
When responding to the user, use a professional tone that prioritizes clarity, without being overly formal.
Use precise language to describe technical concepts. For example, use, "femur" instead of "leg bone" and "cytotoxic T lymphocyte" instead of "killer T cell".
Make your identity as an AI system clear. Don't pretend to be human or include excessive personality, adjectives, or emotional language.

<research_process>
Your goal is to help the user by decomposing questions into sub-topics, generating excellent research plans, u

### 2.4. Provide guidance on final report

In [7]:
system_prompt += """
<final_report>
When generating your final research report, structure it as a comprehensive document that clearly communicates your research findings to the reader. Follow these guidelines:

Report Structure:

- Begin with a concise introduction (1-2 paragraphs) that establishes the research question, explains why it's important, and provides a brief overview of your approach
- Organize the main body into sections that correspond to the major research tasks you completed (e.g., "Literature Review," "Current State Analysis," "Comparative Assessment," "Technical Evaluation," etc.)
- Conclude with a summary section (1-2 paragraphs) that synthesizes key findings and discusses implications

Section Format:

- Write each section in paragraph format using 1-3 well-developed paragraphs
- Each paragraph should focus on a coherent theme or finding
- Use clear topic sentences and logical flow between paragraphs
- Integrate information from multiple sources within paragraphs rather than listing findings separately

Citation Requirements:

- Include proper citations for all factual claims using the format provided in your source materials
- Place citations at the end of sentences before punctuation (e.g., "Recent studies show significant progress in this area .")
- Group related information from the same source under single citations when possible
- Ensure every major claim is supported by appropriate source attribution

Writing Style:

- Use clear, professional academic language appropriate for scientific communication
- Use active voice and strong verbs
- Synthesize information rather than simply summarizing individual sources
- Draw connections between different pieces of information and highlight patterns or contradictions
- Focus on analysis and interpretation, not just information presentation
- Don't use unnecessary words. Keep sentences short and concise.
- WRite for a global audience. Avoid jargon an colloquial language. 

Quality Standards:

- Ensure logical flow between sections and paragraphs
- Maintain consistency in terminology and concepts throughout
- Provide sufficient detail to support conclusions while remaining concise
- End with actionable insights or clear implications based on your research findings </final_report>

</final_report>
"""

print(f'\nThe current system prompt is:\n"""{system_prompt}""""\n')


The current system prompt is:
"""
The current date is October 30, 2025

You are an expert research lead that answers biomedical questions using scientific literature and other authoritative sources. 
You maintain user trust by being consistent (dependable or reliable), benevolent (demonstrating good intent, connectedness, and care), transparent (truthful, humble, believable, and open), and competent (capable of answering questions with knowledge and authority).
When responding to the user, use a professional tone that prioritizes clarity, without being overly formal.
Use precise language to describe technical concepts. For example, use, "femur" instead of "leg bone" and "cytotoxic T lymphocyte" instead of "killer T cell".
Make your identity as an AI system clear. Don't pretend to be human or include excessive personality, adjectives, or emotional language.

<research_process>
Your goal is to help the user by decomposing questions into sub-topics, generating excellent research plans, u

## 3. Update Model Configuration

From this point onward, our agent is going to start generating a lot of tokens. We'll need to adjust some configurations to handle this.

### 3.1. Max Tokens

Our agent uses tokens both to reason about user requests and also generate the research plan and report. We need to increase the `max_tokens` limit to avoid exceptions.

In [8]:
from strands.models import BedrockModel

model = BedrockModel(
    model_id=MODEL_ID,
    max_tokens=10000,  # Increase the max tokens to accomodate the research plan and report generation
)

### 3.2. Prompt Caching

Prompt caching is an optional feature you can use with supported models on Amazon Bedrock to reduce inference latency and input token costs. It that allows you to cache long and repeated contexts that are frequently reused for multiple queries. When you cache portions of your prompt, the model can skip recomputation of those inputs, leading to faster responses and lower costs. This is particularly useful for scenarios like chatbots where users upload documents and ask multiple questions about them.

For Claude models, Amazon Bedrock offers simplified cache management that automatically checks for cache hits at previous content block boundaries, looking back up to approximately 20 content blocks from your specified breakpoint. This reduces the complexity of manually placing cache checkpoints. 

This feature is particularly valuable for applications with repetitive, long contexts where the same information is processed multiple times across different user interactions.

In [9]:
from strands.models import BedrockModel

model = BedrockModel(
    model_id=MODEL_ID,
    max_tokens=10000,
    cache_prompt="default",  # Enables caching of the system prompt and (future) tool definitions
)

### 3.3. Extended / Interleaved Thinking

Extended thinking is a powerful feature that gives LLMs like Claude Sonnet enhanced reasoning capabilities for complex tasks by allowing it to work through problems step-by-step before delivering a final answer. When extended thinking is enabled, Claude creates internal reasoning blocks where it outputs its thought process, then incorporates insights from this reasoning to craft a more thoughtful final response. 

Interleaved thinking is a particularly sophisticated feature available in Claude 4 models that enables Claude to think between tool calls and make more nuanced decisions based on intermediate results. This specifically enables:

- Dynamic Reasoning: Claude can reason about tool results before deciding what to do next
- Chain Tool Calls: Multiple tool calls can be linked with reasoning steps in between
- Adaptive Planning: The model can adjust its approach dynamically based on intermediate results 

The main difference you'll notice with the interleaved thinking is that Event loop is acting on LLM's "thoughts", rather than "decisions".  In a traditional event loop, the thoughts are hidden. We have to wait until LLM renders either a decision to call a tool or produces the Final Answer. 

In case of interleaved thinking, LLM is "leaking" its thoughts into the even loop while it's still in that second step - "LLM is thinking" - and event loop is configured to executed the tools as soon as LLM "thinks" about doing it. What this means is that by the time LLM is done thinking, it actually has the Final Answer, on the very first "decision". 

In [10]:
from strands.models import BedrockModel

model = BedrockModel(
    model_id=MODEL_ID,
    max_tokens=10000,
    cache_prompt="default",
    temperature=1,  # Required to be 1 when thinking is enabled
    additional_request_fields={
        # Enable interleaved thinking beta feature
        "anthropic_beta": ["interleaved-thinking-2025-05-14"],
        # Configure reasoning parameters
        "reasoning_config": {
            "type": "enabled",  # Turn on thinking
            "budget_tokens": 3000,  # Thinking token budget
        },
    },
)

Now our model should have sufficient token capacity to complete the mock planning and report generation process. It still doesn't have access to any tools, so the information presented will only be from it's own knowledge and likely contain inaccuracies!

In [11]:
from strands import Agent

agent = Agent(model=model, system_prompt=system_prompt)
response = agent("How safe and effective are GLP-1 drugs for long term use?")
response = agent(
    "Please proceed with the first section only and then immediately draft the final report. You may proceed when ready."
)

response.metrics.accumulated_usage

Let me analyze this question following the research process outlined:

1. **Assess the question**: 
The user is asking about GLP-1 drugs (glucagon-like peptide-1 receptor agonists), specifically focusing on:
- Safety profile for long-term use
- Effectiveness for long-term use
- This covers medications like semaglutide (Ozempic, Wegovy), liraglutide (Saxenda, Victoza), tirzepatide (Mounjaro, Zepbound), etc.

Key entities: GLP-1 receptor agonists, safety, effectiveness, long-term use
Main concepts: pharmacovigilance data, clinical trial results, real-world evidence
Specific facts needed: adverse events, efficacy data, duration of studies, patient populations
Temporal constraints: "long-term" - need to define what constitutes long-term and what data is available

What the user likely cares about most: Understanding the risk-benefit profile for extended use, potential side effects that emerge over time, sustained efficacy, and whether these drugs are safe for chronic use.

The answer would

{'inputTokens': 870,
 'outputTokens': 3869,
 'totalTokens': 8951,
 'cacheReadInputTokens': 2106,
 'cacheWriteInputTokens': 2106}

## 4. Full Deep Research Agent with tools

We want our agent to create and update files throughout the research process, so we'll need to give it the tools to do so. Our full deep research agent will require access to three tools:

1. `search_pmc_tool` from notebook 1 for searching PubMed Central (PMC) for relevant scientific articile abstracts
2. `gather_evidence_tool` from notebook 2 for gathering detailed evidence from full-text articles using the PaperQA2 tool
3. The pre-built Strands `editor` tool for writing and updating files.

Let's add them to our agent and start a test run. This will take several minutes to complete.

In [12]:
from strands import Agent
from strands.models import BedrockModel
from strands_tools import editor
from search_pmc import search_pmc_tool
from gather_evidence import gather_evidence_tool
import os

model = BedrockModel(
    model_id=MODEL_ID,
    max_tokens=10000,
    cache_prompt="default",
    temperature=1,
    additional_request_fields={
        "anthropic_beta": ["interleaved-thinking-2025-05-14"],
        "reasoning_config": {
            "type": "enabled",
            "budget_tokens": 3000,
        },
    },
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[editor, search_pmc_tool, gather_evidence_tool],
)
response = agent("How safe and effective are GLP-1 drugs for long term use?")
response = agent(
    "Please proceed with researching the first section only and then immediately draft the final report. You may proceed when ready."
)

response.metrics.accumulated_usage

Let me analyze this question about GLP-1 drugs and their safety and effectiveness for long-term use following the research process outlined.

1. **Assess the question**: The user is asking about GLP-1 drugs, specifically their safety and effectiveness for long-term use. This involves:
- Main concepts: GLP-1 agonists, safety profile, efficacy, long-term outcomes
- Key entities: Specific GLP-1 drugs (semaglutide, liraglutide, etc.), clinical trials, adverse events, long-term studies
- Relationships: Drug mechanisms, patient outcomes, risk-benefit profiles
- Temporal constraints: Focus on long-term use (likely >1 year)
- Important features: The user likely cares about both safety (side effects, contraindications) and effectiveness (weight loss, diabetes control, cardiovascular outcomes) with emphasis on sustained use
- Expected answer form: A comprehensive analysis covering both safety and efficacy data from clinical studies

2. **Determine the question type**: This appears to be a **deep

INFO | search_pmc | Searching PMC for: GLP-1 receptor agonists mechanism of action pharmacology AND last 5 years[dp]
INFO | search_pmc | Found 100 article pmcids
INFO | search_pmc | Fetching 100 PM articles
INFO | search_pmc | Successfully fetched 100 articles
INFO | search_pmc | Calculating citation relationships and ranking articles
INFO | search_pmc | Citation ranking completed successfully
INFO | search_pmc | Applied max_filtered_result_count limit: returning 10 of 100 articles
INFO | search_pmc | Top 3 results:
--------------------------------------------------
Title: GLP-1 Receptor Agonists: Beyond Their Pancreatic Effects
PMID: 34497589
PMC: PMC8419463
DOI: 10.3389/fendo.2021.721135
Source: https://doi.org/10.3389/fendo.2021.721135
Authors: Xin Zhao, Minghe Wang, Zhitong Wen, Zhihong Lu, Lijuan Cui, Chao Fu, Huan Xue, Yunfeng Liu, Yi Zhang
Journal: Frontiers in Endocrinology
Year: 2021
Abstract: Glucagon like peptide-1 (GLP-1) is an incretin secretory molecule. GLP-1 receptor ag

Now let me search for more specific information about long-term safety and efficacy of GLP-1 drugs to complement the mechanism information.

Tool #3: search_pmc_tool


INFO | search_pmc | Searching PMC for: GLP-1 receptor agonists long-term safety efficacy diabetes obesity cardiovascular outcomes AND last 3 years[dp]
INFO | search_pmc | Found 100 article pmcids
INFO | search_pmc | Fetching 100 PM articles
INFO | search_pmc | Successfully fetched 100 articles
INFO | search_pmc | Calculating citation relationships and ranking articles
INFO | search_pmc | Citation ranking completed successfully
INFO | search_pmc | Applied max_filtered_result_count limit: returning 10 of 100 articles
INFO | search_pmc | Top 3 results:
--------------------------------------------------
Title: Combining glucagon-like peptide-1 receptor agonists (GLP-1RAs) and sodium-glucose cotransporter-2 inhibitors (SGLT2is) in patients with type 2 diabetes mellitus (T2DM)
PMID: 37005640
PMC: PMC10067319
DOI: 10.1186/s12933-023-01798-4
Source: https://doi.org/10.1186/s12933-023-01798-4
Authors: Pierre Gourdy, Patrice Darmon, François Dievart, Jean-Michel Halimi, Bruno Guerci
Journal: Car

Now let me gather some specific information about long-term safety and efficacy from one of the most relevant papers I found.

Tool #4: gather_evidence_tool


INFO | gather_evidence | Starting gather_evidence for PMCID: PMC8419463, question: What are the long-term safety concerns and adverse effects of GLP-1 receptor agonists? What is the evidence for their long-term efficacy?
INFO | gather_evidence | Attempting to download s3://pmc-oa-opendata/oa_comm/txt/all/PMC8419463.txt to my_papers/PMC8419463/txt/PMC8419463.txt
INFO | gather_evidence | Successfully downloaded file to my_papers/PMC8419463/txt/PMC8419463.txt
INFO | gather_evidence | Successfully retrieved commercial article PMC8419463
INFO | gather_evidence | Processing paper with paper-qa for question: What are the long-term safety concerns and adverse effects of GLP-1 receptor agonists? What is the evidence for their long-term efficacy?
INFO | gather_evidence | Invoking paper-qa
INFO | gather_evidence | Successfully answered question for PMC8419463


Now let me update the outline with the research I've completed and create the final report.

Tool #5: editor
Now I'll create the final report based on my research findings. Since the user asked me to only research the first section and then draft the final report, I'll write a comprehensive report that includes the mechanisms information I gathered as well as my existing knowledge about GLP-1 drugs' long-term safety and efficacy to provide a complete answer to their question.

Tool #6: editor
I've completed the research and created the final report on the safety and effectiveness of GLP-1 drugs for long-term use. The report addresses both aspects of your question:

**Safety for Long-Term Use:**
- Generally favorable safety profile supported by extensive clinical trial data
- Common gastrointestinal side effects (nausea, vomiting) that typically diminish over time
- Low risk of hypoglycemia due to glucose-dependent mechanism
- Monitoring required for rare risks like pancreatitis and thy

{'inputTokens': 45561,
 'outputTokens': 5235,
 'totalTokens': 94612,
 'cacheReadInputTokens': 32862,
 'cacheWriteInputTokens': 10954}

In this case, we limited the research to only section 1. However, the agent was able to retrieve relevant information and generate a high-quality report with citations. 

## 5. Deploy to Amazon Bedrock AgentCore Runtime

Let's look at the new agent definition

In [13]:
%pycat agent.py

import asyncio

from bedrock_agentcore.runtime import BedrockAgentCoreApp
from search_pmc import search_pmc_tool
from gather_evidence import gather_evidence_tool
from strands import Agent
from strands.models import BedrockModel
from strands_tools import editor
import os

from config import MODEL_ID, SYSTEM_PROMPT

app = BedrockAgentCoreApp()

model = BedrockModel(
    model_id=MODEL_ID,
    max_tokens=20000,
    cache_prompt="default",
    temperature=1,
    additional_request_fields={
        "anthropic_beta": ["interleaved-thinking-2025-05-14"],
        "reasoning_config": {
            "type": "enabled",
            "budget_tokens": 3000,
        },
    },
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

agent = Agent(
    model=model,
    tools=[editor, search_pmc_tool, gather_evidence_tool],
    system_prompt=SYSTEM_PROMPT,
)


@app.entrypoint
async def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    prin

In [14]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

ssm = boto3.client("ssm")

agentcore_runtime = Runtime()
agentcore_runtime.configure(
    agent_name="pmc_deep_research_agent",
    auto_create_ecr=True,
    execution_role=ssm.get_parameter(
        Name="/deep-research-workshop/agentcore-runtime-role-arn"
    )["Parameter"]["Value"],
    entrypoint="agent.py",
    memory_mode="NO_MEMORY",
    requirements_file="requirements.txt",
)

Entrypoint parsed: file=/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/03-reason-and-plan/agent.py, bedrock_agentcore_name=agent
INFO | bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint | Entrypoint parsed: file=/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/03-reason-and-plan/agent.py, bedrock_agentcore_name=agent
Memory disabled - agent will be stateless
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore | Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: pmc_deep_research_agent
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Configuring BedrockAgentCore agent: pmc_deep_research_agent


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Memory disabled
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Memory disabled
Network mode: PUBLIC
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Network mode: PUBLIC
Generated .dockerignore
INFO | bedrock_agentcore_starter_toolkit.utils.runtime.container | Generated .dockerignore
Generated Dockerfile: Dockerfile
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Generated Dockerfile: Dockerfile
Generated .dockerignore: /Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/03-reason-and-plan/.dockerignore
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Generated .dockerignore: /Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/03-reason-and-plan/.dockerignore
Setting 'pmc_deep_research_agent' as default agent
INFO | bedrock_agentcore_starter_toolkit.utils.runtime.config | Setting 'pmc_deep_re

ConfigureResult(config_path=PosixPath('/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/03-reason-and-plan/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/03-reason-and-plan/Dockerfile'), dockerignore_path=PosixPath('/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/03-reason-and-plan/.dockerignore'), runtime='None', region='us-west-2', account_id='942514891246', execution_role='arn:aws:iam::942514891246:role/AmazonBedrockAgentCore-us-west-2-06fe41b4be03', ecr_repository=None, auto_create_ecr=True, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

In [15]:
agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore | 🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore |    • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore |    • No local Docker required
💡 Available deployment modes:
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore | 💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore |    • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
INFO | bedrock_agentcore_starter_toolkit.notebook.

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-pmc_deep_research_agent


✅ ECR repository available: 942514891246.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-pmc_deep_research_agent
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.launch | ✅ ECR repository available: 942514891246.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-pmc_deep_research_agent
Using execution role from config: arn:aws:iam::942514891246:role/AmazonBedrockAgentCore-us-west-2-06fe41b4be03
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.launch | Using execution role from config: arn:aws:iam::942514891246:role/AmazonBedrockAgentCore-us-west-2-06fe41b4be03
Preparing CodeBuild project and uploading source...
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.launch | Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: pmc_deep_research_agent
INFO | bedrock_agentcore_starter_toolkit.services.codebuild | Getting or creating CodeBuild execution role for agent: pmc_deep_research_agent
Role name: Amaz

LaunchResult(mode='codebuild', tag='bedrock_agentcore-pmc_deep_research_agent:latest', env_vars=None, port=None, runtime=None, ecr_uri='942514891246.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-pmc_deep_research_agent', agent_id='pmc_deep_research_agent-CH8cNoB6ek', agent_arn='arn:aws:bedrock-agentcore:us-west-2:942514891246:runtime/pmc_deep_research_agent-CH8cNoB6ek', codebuild_id='bedrock-agentcore-pmc_deep_research_agent-builder:232a3361-2db2-40e5-aad3-4fccbdd58190', build_output=None)

In [16]:
%%time

agentcore_runtime.invoke(
    {"prompt": "How safe and effective are GLP-1 drugs for long term use?"}
)

I'll help you research the long

-term safety and effectiveness

of GLP-1

drugs. This

is a complex

...

you have additional

information to share before I

begin the research.

CPU times: user 149 ms, sys: 57.2 ms, total: 206 ms
Wall time: 41.5 s


{}

## 6. (Optional) Interact with agent using AgentCore Chat

Follow these steps to open an interactive chat session with your new agent.

1. Open a command line terminal in your notebook environment.
2. Navigate to the project root folder (where `pyproject.toml` is located).
3. Run `pip install .` to install the workshop tools including the chat CLI.
4. Run `agentcore-chat` to launch the CLI.
5. Select the `pmc_deep_research_agent` by typing its name or index in the terminal and press Enter.
6. Ask your question at the `You:` prompt and press Enter.


## 7. (Optional) Clean Up

Run the next notebook cell to delete the AgentCore runtime environment.

In [ ]:
import boto3

agentcore_client = boto3.client("bedrock-agentcore-control")
agent_status = agentcore_runtime.status()

agentcore_client.delete_agent_runtime(agentRuntimeId=agent_status.config.agent_id)